In [1]:
from __future__ import print_function

import datetime

import numpy as np
import pandas as pd
import quandl


In [5]:
pip install cryptography


     |████████████████████████████████| 4.0 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 446 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 1.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import quandl

# Set your API key
quandl.ApiConfig.api_key = '4wvoLJ6gWmSatjewtybm'

# Specify the futures contract code
# For example, for the WTI Crude Oil Futures (CL) from CME
future_data = quandl.get("CHRIS/CME_CL1")

# Display the data
print(future_data.head())

# You can also save the data to a CSV file
future_data.to_csv("futures_data.csv")


             Open   High    Low   Last  Change  Settle  Volume  \
Date                                                             
1983-03-30  29.01  29.56  29.01  29.40     NaN   29.40   949.0   
1983-03-31  29.40  29.60  29.25  29.29     NaN   29.29   521.0   
1983-04-04  29.30  29.70  29.29  29.44     NaN   29.44   156.0   
1983-04-05  29.50  29.80  29.50  29.71     NaN   29.71   175.0   
1983-04-06  29.90  29.92  29.65  29.90     NaN   29.90   392.0   

            Previous Day Open Interest  
Date                                    
1983-03-30                       470.0  
1983-03-31                       523.0  
1983-04-04                       583.0  
1983-04-05                       623.0  
1983-04-06                       640.0  


In [1]:
import quandl
import pymysql
import pandas as pd

# Set your Quandl API key
quandl.ApiConfig.api_key = '4wvoLJ6gWmSatjewtybm'

# Query the futures data
future_data = quandl.get("CHRIS/CME_CL1")

# Convert the DataFrame index to a column (for example, if the index is the date)
future_data.reset_index(inplace=True)

db = pymysql.connect(
    host="localhost",
    user="securities_master",
    password="1120",
    database="sec_user",
    #auth_plugin='mysql_native_password'  # or 'caching_sha2_password'
)


# Create a cursor object
cursor = db.cursor()

# Define the SQL query to insert data
insert_query = """
INSERT INTO futures_data (date, open, high, low, settle, volume, prev_day_open_interest)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame into the MySQL table
for index, row in future_data.iterrows():
    cursor.execute(insert_query, (row['Date'], row['Open'], row['High'], row['Low'], 
                                  row['Settle'], row['Volume'], row['Prev. Day Open Interest']))

# Commit the transaction
db.commit()

# Close the cursor and connection
cursor.close()
db.close()


OperationalError: (1045, "Access denied for user 'securities_master'@'localhost' (using password: YES)")

In [2]:
def futures_rollover_weights(start_date, expiry_dates, contracts, rollover_days=5):
    """This constructs a pandas DataFrame that contains weights (between 0.0 and 1.0)
    of contract positions to hold in order to carry out a rollover of rollover_days
    prior to the expiration of the earliest contract. The matrix can then be
    'multiplied' with another DataFrame containing the settle prices of each
    contract in order to produce a continuous time series futures contract."""

    # Construct a sequence of dates beginning from the earliest contract start
    # date to the end date of the final contract
    dates = pd.date_range(start_date, expiry_dates[-1], freq='B')

    # Create the 'roll weights' DataFrame that will store the multipliers for
    # each contract (between 0.0 and 1.0)
    roll_weights = pd.DataFrame(np.zeros((len(dates), len(contracts))),
                                index=dates, columns=contracts)
    prev_date = roll_weights.index[0]

    # Loop through each contract and create the specific weightings for
    # each contract depending upon the settlement date and rollover_days
    for i, (item, ex_date) in enumerate(expiry_dates.iteritems()):
        if i < len(expiry_dates) - 1:
            roll_weights.ix[prev_date:ex_date - pd.offsets.BDay(), item] = 1
            roll_rng = pd.date_range(end=ex_date - pd.offsets.BDay(),
                                     periods=rollover_days + 1, freq='B')

            # Create a sequence of roll weights (i.e. [0.0,0.2,...,0.8,1.0]
            # and use these to adjust the weightings of each future
            decay_weights = np.linspace(0, 1, rollover_days + 1)
            roll_weights.ix[roll_rng, item] = 1 - decay_weights
            roll_weights.ix[roll_rng, expiry_dates.index[i+1]] = decay_weights
        else:
            roll_weights.ix[prev_date:, item] = 1
        prev_date = ex_date
    return roll_weights

In [4]:
if __name__ == "__main__":
    # Download the current Front and Back (near and far) futures contracts
    # for WTI Crude, traded on NYMEX, from Quandl.com. You will need to 
    # adjust the contracts to reflect your current near/far contracts 
    # depending upon the point at which you read this!
    wti_near = quandl.get("OFDP/FUTURE_CLF2014")
    wti_far = quandl.get("OFDP/FUTURE_CLG2014")
    wti = pd.DataFrame({'CLF2014': wti_near['Settle'],
                        'CLG2014': wti_far['Settle']}, index=wti_far.index)

    # Create the dictionary of expiry dates for each contract
    expiry_dates = pd.Series({'CLF2014': datetime.datetime(2013, 12, 19),
                              'CLG2014': datetime.datetime(2014, 2, 21)}).order()

    # Obtain the rollover weighting matrix/DataFrame
    weights = futures_rollover_weights(wti_near.index[0], expiry_dates, wti.columns)

    # Construct the continuous future of the WTI CL contracts
    wti_cts = (wti * weights).sum(1).dropna()

    # Output the merged series of contract settle prices
    print(wti_cts.tail(60))

NotFoundError: (Status 410) (Quandl Error QECx06) API requests for database 'OFDP' have been deprecated. Please contact clientsuccess@nasdaq.com for further information.

In [5]:
import datetime
import yfinance as yf

if __name__ == "__main__":
    spy = yf.download(
        "SPY",
        start="2007-01-01",
        end="2015-06-15"
    )
    print(spy.tail())


[*********************100%%**********************]  1 of 1 completed


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2015-06-08  209.639999  209.820007  208.389999  208.479996  177.070801   
2015-06-09  208.449997  209.100006  207.690002  208.449997  177.045364   
2015-06-10  209.369995  211.410004  209.300003  210.949997  179.168716   
2015-06-11  211.479996  212.089996  211.199997  211.630005  179.746292   
2015-06-12  210.639999  211.479996  209.679993  210.009995  178.370316   

               Volume  
Date                   
2015-06-08   89063300  
2015-06-09  105034700  
2015-06-10  134551300  
2015-06-11   73876400  
2015-06-12  135382400  
